# LangChain Use-Case Example: AI API Multiplexer

This example shows how you can connect to many different AI APIs using the same code by going through LangChain.

You can run this code if you have an OpenAI API key.  If you have other API keys then you can run some of the demos that use multiple AI APIs from the same code.

## Setup code

The first code cell installs the LangChain modules and retrieves AI API credentials.  Please set up Colab secrets for any of these:

* `OPENAI_API_KEY`    -- To use the OpenAI API
* `AWS_ACCESS_KEY_ID` -- To use AWS Bedrock
* `AWS_SECRET_ACCESS_KEY`
* `AWS_REGION_NAME`

In [7]:
# Load secrets
import os
from dotenv import load_dotenv
load_dotenv()

def load_environment_variables(variable_names):
    for var_name in variable_names:
        if var_name not in os.environ:
            try:
                from google.colab import userdata
                value = userdata.get(var_name)
                if value:
                    os.environ[var_name] = value
            except ImportError:
                pass
        if var_name not in os.environ:
            raise ValueError(f"{var_name} not found. Please set it in .env file or Google Colab secrets.")
        else:
            print(f"Successfully loaded {var_name} from environment variables.")

variables_to_load = [
    "OPENAI_API_KEY",
    "AWS_ACCESS_KEY_ID",
    "AWS_SECRET_ACCESS_KEY",
    "AWS_REGION_NAME"
]

load_environment_variables(variables_to_load)

Successfully loaded OPENAI_API_KEY from environment variables.
Successfully loaded AWS_ACCESS_KEY_ID from environment variables.
Successfully loaded AWS_SECRET_ACCESS_KEY from environment variables.
Successfully loaded AWS_REGION_NAME from environment variables.


In [3]:
# Install necessary libraries
!pip install langchain langchain-community langchain-core langchain-openai

# LangChain setup: Show more about what's happening as it happens.
from langchain.globals import set_debug
set_debug(True)

## Example: Minimal AI API Request

Here's a simple OpenAI API request with LangChain.

If you do this same request [with the OpenAI API](https://platform.openai.com/docs/guides/chat-completions/overview) then you get the response in `message = completion.choices[0].message.content`.  LangChain encapsulates those details so that you can think about what you're doing instead of working to accommodate the API.

In [9]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name="gpt-4o")

model.invoke("Is the Sagrada Família located in Barcelona?")

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Is the Sagrada Família located in Barcelona?"
  ]
}
[llm/end] [llm:ChatOpenAI] [1.22s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Yes, the Sagrada Família is located in Barcelona, Spain. It is a large Roman Catholic basilica designed by the famous architect Antoni Gaudí. Construction of the Sagrada Família began in 1882 and is still ongoing, making it one of the most famous and enduring architectural projects in the world. The basilica is renowned for its unique and intricate design, blending Gothic and Art Nouveau styles, and it is one of Barcelona's most iconic landmarks.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messa

AIMessage(content="Yes, the Sagrada Família is located in Barcelona, Spain. It is a large Roman Catholic basilica designed by the famous architect Antoni Gaudí. Construction of the Sagrada Família began in 1882 and is still ongoing, making it one of the most famous and enduring architectural projects in the world. The basilica is renowned for its unique and intricate design, blending Gothic and Art Nouveau styles, and it is one of Barcelona's most iconic landmarks.", response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 16, 'total_tokens': 107}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3aa7262c27', 'finish_reason': 'stop', 'logprobs': None}, id='run-2550873c-5e3d-492b-9410-5d29b9b44934-0', usage_metadata={'input_tokens': 16, 'output_tokens': 91, 'total_tokens': 107})

## Example: Two Sequential Requests, Two Different Models

This example uses two different models from the same OpenAI API to handle two different parts of a larger task.

In [10]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# First LLM: Use gpt-4o for the CoT prompt to get a detailed answer
cot_prompt_template = """
Question: {question}
Please provide a detailed explanation before answering whether the statement is true or false.
"""

cot_prompt = PromptTemplate(
    template=cot_prompt_template,
    input_variables=["question"])

# Create the first chain using OpenAI's gpt-4o model with a CoT prompt
chain_1 = ChatOpenAI(temperature=0.7, model_name="gpt-4o")

# Example question
question = "Is the Eiffel Tower located in Paris?"

# Invoke the first chain
cot_response = chain_1.invoke(cot_prompt.format(question=question))
detailed_answer = cot_response.content
print("Detailed answer:", detailed_answer)

# Second LLM: Use gpt-4o-mini to categorize the detailed answer as 'Yes' or 'No'
yes_no_prompt_template = """
Based on the following answer, please categorize it strictly as either "Yes" or "No":
Answer: "{detailed_answer}"
"""

yes_no_prompt = PromptTemplate(
    template=yes_no_prompt_template,
    input_variables=["detailed_answer"])

# Create the second chain using OpenAI's gpt-4o-mini model for yes/no clarification
chain_2 = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

# Invoke the second chain
yes_no_response = chain_2.invoke(yes_no_prompt.format(detailed_answer=detailed_answer))
yes_no_answer = yes_no_response.content.strip()
print("Yes/No Answer:", yes_no_answer)

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: \nQuestion: Is the Eiffel Tower located in Paris?\nPlease provide a detailed explanation before answering whether the statement is true or false."
  ]
}
[llm/end] [llm:ChatOpenAI] [5.10s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Yes, the statement is true; the Eiffel Tower is located in Paris.\n\nLet's delve into some details to provide a comprehensive explanation:\n\n1. **Historical Context**: The Eiffel Tower was constructed between 1887 and 1889 as the entrance arch for the 1889 Exposition Universelle (World's Fair), which was held to celebrate the 100th anniversary of the French Revolution.\n\n2. **Geographical Location**: The Eiffel Tower is situated on the Champ de Mars, a large public greenspace in the 7th arrondissement of Paris, France. The exact coordinates are approximately 48.8584° N latitude and 2.2945° E longitude.\n\n3. **Architectural Significan